<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 1 - Búsqueda de playlists</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---
<br>

<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>


* [1. Introducción](#section1)
* [2. Librería _Spotipy_](#section2)
* [3. Métodos auxiliares](#section3)
* [4. Método para la búsqueda de playlists](#section4)
* [4. Método para procesar las listas de palabras](#section5)
* [6. Proceso de descarga de playlists](#section6)

<br>

---

In [1]:
# Permite establecer la anchura de la celda
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
#!pip install tqdm ipywidgets
#!jupyter nbextension enable --py widgetsnbextension
#!pip install requests
#!pip install spotipy

In [3]:
import codecs
import csv
import datetime
import os
import requests
import shutil
import spotipy
import time

from collections import defaultdict
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm.notebook import tqdm as tqdm_nb

---
<br>

<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

En esta libreta nos encargaremos de buscar playlists para su posterior descarga. En concreto, vamos a obtener los datos básicos para analizarlos y quedarnos con un conjunto de listas que descargar. Para la búsqueda, partiremos de las 3.000 palabras más frecuentes usadas en inglés que hemos obtenido de ["*EF Education First*"](https://www.ef.com/wwen/english-resources/english-vocabulary/top-3000-words/).

También se han creado varios conjuntos que contienen los días de la semana, meses, eventos especiales, géneros musicales, períodos musicales, estados de ánimo, actividades, lugares, etc... Estos conjuntos se encuentran en:

- `split_lists\splitlist-ages.txt`
- `split_lists\splitlist-genres.txt`
- `split_lists\splitlist-special1.txt`
- `split_lists\splitlist-special2.txt`



<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Librería *Spotipy*</font>

<br>

Para trabajar de forma más cómoda con la [WebAPI de Spotify](https://developer.spotify.com/documentation/web-api/), haremos uso de la librería [`Spotipy`](https://spotipy.readthedocs.io).

_Spotipy_ es una biblioteca de _Python_ con la cual obtenemos acceso completo a todos los datos de música proporcionados por la plataforma de _Spotify_ y es compatible con todas las funciones de la WebAPI, incluido el acceso a todos los endpoints y el soporte a la autorización por parte del usuario (para algunas características se requiere que el usuario confirme que desea otorgar el acceso).

Antes de comenzar a utilizar la librería *Spotipy*, establecemos las claves que hemos obtenido al crear un proyecto en el [_Spotify Developer Dashboard_](https://developer.spotify.com/dashboard). Por último, creamos el gestor que utilizaremos para buscar listas de reproducción.

In [4]:
sp_client_id = '' #SpotifyClientID
sp_client_secret = '' #SpotifyClientSecret

# Crea el gestor
client_credentials_manager = SpotifyClientCredentials(client_id=sp_client_id, client_secret=sp_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Métodos auxiliares</font>
<br>

En este apartado se encuentran los métodos auxiliares que se van a emplear para la búsqueda de playlists, para el procesado de las listas de palabras y para la obtención de la información de las playlists.

In [5]:
# Variables globales

# Directorio donde se almacenan los sets generados
SETS_PATH = 'sets'

# Directorio donde se encuentran las listas de palabras que se generan
SPLIT_LISTS_PATH = 'split_lists'

# Directorio que empleamos para guardar copias de seguridad
BACKUP_PATH = 'backup'

<br>

<a id="section31"></a>
### <font color="#B20033">Bot de Telegram</font>
<br>

Se ha creado un Bot de [_Telegram_](https://www.telegram.org/) para seguir el proceso de búsqueda y recibir notificaciones en caso de que surja cualquier problema.


*Fuente*: [How to create a Telegram bot, and send messages with Python (Medium)](https://medium.com/@ManHay_Hong/how-to-create-a-telegram-bot-and-send-messages-with-python-4cf314d9fa3e).

In [6]:
tl_bot_token = ''
tl_bot_chatID = ''

def telegram_bot_sendtext(bot_message, prefix=''):
    """
    :param bot_message: Mensaje que publicará el bot.
    :param prefix: Prefijo con que se identificará el mensaje (opcional).
    :return: Resultado de la petición.
    """
    bot_token = tl_bot_token
    bot_chatID = tl_bot_chatID
    
    if prefix != '':
        bot_message = '[{}]: {}'.format(prefix,bot_message)
        
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&text=' + bot_message

    response = requests.get(send_text)

    return response.json()['ok']

<br>
<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: El argumento `prefix` lo utilizaremos para distinguir desde dónde se ha enviado el mensaje, en caso de que se empleen varios equipos o procesos para la búsqueda de playlists.
</div>

<br>

Para comprobar que el Bot funciona correctamente:

In [7]:
#test = telegram_bot_sendtext('Testing Telegram Bot')
#print(test)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section32"></a>
### <font color="#B20033">Creación de logs</font>
<br>

Para almacenar la información del proceso de búsqueda, hemos creado una función que nos permitirá guardar aquellos eventos que consideremos de interés junto a la fecha y hora en la que han ocurrido. El nombre del fichero, por defecto, es `log.txt`.

In [8]:
# Escribe un texto en un fichero de logs
def write_to_log(text,file_name='log.txt'):
    """
    :param text: Mensaje a escribir en el log.
    :param file_name: Nombre del fichero donde almacenaremos el log.
    """
    current_date = datetime.datetime.now()
    with open(file_name, 'a') as fp:
        fp.write(f'({current_date}):\t{text}\n')

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section33"></a>
### <font color="#B20033">Listas de palabras</font>
<br>

Para la búsqueda de playlists, los términos con los que vamos a realizar las consultas serán almacenados en ficheros `.txt`, siendo agrupados por su letra inicial (salvo el caso de los términos especiales que hemos indicado en el apartado de '[Introducción](#section1)').

Esto se realiza así para que el proceso de búsqueda pueda hacerse de forma escalonada y también pueda realizarse desde varias máquinas o en varios procesos.

También utilizaremos estos métodos para modificar los ficheros conforme avance el proceso de búsqueda. Así en caso de que surja cualquier problema, aquellas palabras que ya se han tratado no volverán a procesarse cuando se reanude la búsqueda.

In [9]:
# Devuelve una lista con las palabras contenidas en un fichero
def load_word_list(file_name):
    """
    :param file_name: Nombre del fichero que contiene la lista de palabras.
    :return: Lista con las palabras contenidas en el fichero.
    """
    with open(file_name) as file:
        content = file.readlines()
        # Elimina espacios en blanco como `\n` al final de cada línea
        return [word.strip().lower() for word in content] 

In [10]:
# Almacena una lista de palabras en un fichero
def store_word_list(file_path, word_list):
    """
    :param file_path: Ruta donde guardaremos las listas de palabras.
    :param word_list: Lista de palabras a guardar.
    """
    with open(file_path, 'w') as f:
        for word in word_list:
            f.write("%s\n" % word)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section34"></a>
### <font color="#B20033">Diccionarios</font>
<br>

Para agrupar las palabras por su letra inicial, nos apoyaremos en el uso de [_diccionarios_](https://docs.python.org/2/tutorial/datastructures.html#dictionaries).

Los siguientes métodos nos permiten crear diccionarios, con los cuales agrupamos las palabras por letra inicial, y almacenarlos en ficheros:

In [11]:
# Crea un diccionario con la letra inicial como clave
# y cuyo valor es la lista de palabras que comienzan 
# por esa letra
def create_dictionary(input):
    """
    :param input: Lista de palabras.
    :return: Diccionario cuya clave es la letra inicial y sus valores
             la lista de palabras que empiezan dicha letra.
    """
    result = defaultdict(list)
    for word in input:
        result[word[0]].append(word)
    return result

In [12]:
# Convierte el diccionario en ficheros
def split_dict_to_files(dictionary, dir_path, name ='',):
    """
    :param dictionary: Diccionario que contiene una lista de palabras.
    :param name: (Opcional) Prefijo que añadimos al nombre del fichero para identificarlo.
    :param dir_path: Directorio donde se almacenará el fichero resultante.
    """
    
    # Cambio en la nomenclatura en caso 
    # de indicar un prefijo (name)
    if name != '' : 
        name = f'-{name}_'
    else:
        name = '-'
    
    # key   -> Letra 
    # value -> Lista de palabras
    for key, value in dictionary.items():
        file_name = f'splitlist{name}{key}.txt'
        
        # Creamos el directorio donde se almacenarán
        # las listas de palabras que se han generado
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        
        # Guardamos las palabras contenidas en la lista 'value'
        with open(os.path.join(dir_path, file_name), 'w') as file:
            for item in value:
                file.write("%s\n" % item)

<br>

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Ambos métodos ***split_dict_to_files*** y ***store_word_list*** se encargan de almacenar conjuntos de palabras en ficheros `.txt`, pero el primero se utiliza para generar por primera vez los ficheros que contienen las palabras agrupadas por letra inicial y el segundo se utiliza únicamente para almacenar un fichero de palabras en caso de querer eliminar una palabra que ya hemos procesado (modificación).
</div>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

### <a id="section4"></a>
## <font color="#92002A">4 - Método para la búsqueda de playlists</font>

<br>

El siguiente método es el que vamos a emplear para buscar las playlists que contengan el término que indiquemos. Su funcionamiento es el siguiente:
1. Como la WebAPI de _Spotify_ no devuelve todos los resultados en una única llamada, nos encargamos de realizar varias llamadas para recabar todos los resultados que contengan el término solicitado.
2. _Spotify_ limita los resultados de búsqueda a 10.000, por lo tanto, comprobamos que el índice del primer elemento que va a devolver la siguiente llamada (*query_offset*) sea menor que dicho número. Este valor se incrementa al final del bucle.
3. Establecemos el número de items a devolver por llamada a 50, ya que es el máximo permitido.
4. Si una lista es de *Spotify*, no la incluimos en los resultados (ya que estamos interesados en las playlists creadas por usuarios).
5. Tras procesar los resultados de la petición actual, obtenemos la URL de la siguiente petición. En caso de que no existan más resultados, el valor de la siguiente llamada será _null_. Por lo tanto, paramos el proceso y devolvemos los resultados.

In [13]:
# Busca un término en Spotify
def playlist_search(term):
    """
    :param term: Termino con el que se realizará la búsqueda.
    :return: Lista que contiene las playlists obtenidas.
    """
    query_offset = 0 # Contiene el índice del primer elemento a devolver
    nxt = '' # URL de la siguiente página de items (null en caso de no haber)
    limit = 50 # Limite de items en la respuesta
    playlists = list()
    
    # La API de Spotify limita los resultados a 10000,
    # con lo cual comprobamos que el 'query_offset' sea
    # menor a dicho número
    while (query_offset < 10000) and (nxt != None):
        query = sp.search(term, limit, type='playlist', offset=query_offset)
        results = query['playlists']['items']
        for result in results:
            # Descartamos las listas del usuario 'Spotify'
            if str(result['owner']['display_name']).lower() != 'spotify':
                playlists.append({'id' : result['id'],
                                  'num_tracks' : result['tracks']['total'],
                                  'name': result['name']})
        nxt = query['playlists']['next']
        query_offset += limit
        time.sleep(0.7) # Retardo
        
    return playlists

<br>

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Como podemos observar, en la mayoría de los métodos que implican llamadas a la WebAPI de _Spotify_ hemos introducido un retardo para no saturar el servidor realizando miles de peticiones consecutivas. Aparte de evitar cualquier saturación, si no lo hiciéramos infringiríamos uno de los [_Términos del Servicio (ToS)_](https://developer.spotify.com/terms/#iv) de _Spotify_.
</div>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section5"></a>
## <font color="#92002A">5 - Método para procesar las listas de palabras</font>
<br>

El método ***process_split_lists*** es el encargado de procesar todas las listas de términos que hemos creado y que se encuentran almacenadas dentro de un directorio:

In [14]:
# Procesa los ficheros con las palabras
def process_split_lists(sets_path, split_lists_path):
    """
    :param sets_path: Ruta donde almacenaremos los resultados obtenidos.
    :param split_lists_path: Ruta de donde se encuentran las listas de palabras.
    """
    
    # Creamos el fichero donde se almacenaran los sets con
    # los resultados de las búsquedas en caso de no existir
    if not os.path.exists(sets_path):
        os.makedirs(sets_path)
    
    # Definimos las barras de progreso que vamos a emplear
    # para la búsqueda de playlists
    pbar_general = tqdm_nb(total=len(os.listdir(split_lists_path)))
    pbar_general.set_description('Ficheros procesados')
    pbar_file = tqdm_nb(total=1) # Nota: Si no se establece un valor superior a 0,
                                 #       la barra no se muestra correctamente (bug)
    try:
        for file_name in os.listdir(split_lists_path):
            file_path = os.path.join(split_lists_path,file_name)
            
            # Comprobamos que es un fichero y comienza por 'split_lists_'
            # (prefijo utilizado para las listas de palabras)
            if os.path.isfile(file_path) and 'splitlist-' in file_name:
                words_list = load_word_list(file_path)
                pending_words = words_list.copy()
                pbar_file.set_description(f"Palabras en '{file_name}'")
                pbar_file.reset(total=len(words_list))
                
                # Extraemos el identificador del nombre del fichero
                # y establecemos la ruta del fichero .csv
                set_id = file_name.split('-')[-1].split('.')[0]
                set_file_path = os.path.join(sets_path,f'set-{set_id}.csv')
                
                # Para cada palabra del fichero realizamos la búsqueda
                for index, word in enumerate(words_list):
                    result = playlist_search(word)
                    
                    # Cuando se termina la búsqueda de la palabra,
                    # almacenamos los resultados en el fichero .csv
                    with codecs.open(set_file_path, 'a', 'utf-8') as csvfile:
                        writer = csv.DictWriter(csvfile, fieldnames=['id','name','num_tracks'])
                        for data in result:
                            writer.writerow(data)

                    # Eliminamos la palabra procesada para que en caso de que se produzca
                    # un error, cuando volvamos a procesar el fichero no se vuelvan a
                    # buscar aquellas palabras que ya han sido procesadas
                    pending_words.pop(0)
                    store_word_list(file_path, pending_words)
                    
                    # Actualizamos la barra de progreso para las palabras
                    # del fichero actual
                    pbar_file.update()
                    time.sleep(30) # Retardo
                
                # El fichero ha terminado de procesarse
                message = f'{file_name} file has been processed.'
                write_to_log(message)
                telegram_bot_sendtext(message)    
                
                # Una vez procesado el fichero, lo borramos
                os.remove(file_path)
                time.sleep(60) # Retardo      
            
            # Actualizamos la barra para los ficheros leidos
            pbar_general.update()
        
        # Una vez procesados todos los ficheros, borramos el directorio
        os.rmdir(split_lists_path)        
        
        message = 'Search process completed.'
        write_to_log(message)
        telegram_bot_sendtext(message)  
    except Exception as e:
        # Guardamos el error en el log y avisamos por Telegram
        message = f'Something went wrong in "process_split_lists". Retrying in 5 minutes.'
        write_to_log(message)
        write_to_log(str(e))
        telegram_bot_sendtext(message)
        
        # Esperamos 5 minutos antes de repetir el proceso
        time.sleep(300)
        process_split_lists(sets_path)

<br>

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Los tiempos indicados en las barras de progreso son orientativos, puesto que cada fichero contiene un número diferente de términos y cada término requiere un tiempo distinto.
</div>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section6"></a>
## <font color="#92002A">6 - Proceso de obtención de playlists</font>
<br>

Una vez hemos definido todos los métodos que vamos a utilizar, empezamos con el proceso de búsqueda de playlists:

In [15]:
# Cargamos la lista con las 3000 palabras más usadas
eng_words = load_word_list('files/list-words_eng.txt')

# Creamos un diccionario con las palabras agrupadas por letra inicial
eng_dict = create_dictionary(eng_words)

# Convertimos el diccionario en ficheros de texto
split_dict_to_files(eng_dict, SPLIT_LISTS_PATH, name='eng')

In [16]:
# Procesamos los ficheros
process_split_lists(SETS_PATH, SPLIT_LISTS_PATH)

<br>

Tras completarse el proceso, guardamos una copia del directorio con los conjuntos obtenidos en un fichero comprimido y lo movemos al directorio establecido en la variable `BACKUP_PATH`:

In [17]:
if not os.path.exists(BACKUP_PATH):
    os.makedirs(BACKUP_PATH)

shutil.make_archive('sets', 'zip', SETS_PATH)
shutil.move('sets.zip', BACKUP_PATH)

'backup/sets.zip'

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>